In [1]:
from new_basis_llp_qaoa.statevector_sim import CYPStateVector 
from new_basis_llp_qaoa.statevector_sim.cyp_statevector_sim import CYPStatevectorSim
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from scipy import sparse
# Q-Learning values. (Discounted Payoff action is in first column).
Qs = sparse.load_npz('data/Qs.npz').toarray()

# Association matrix
As = sparse.load_npz('data/As.npz').toarray()

# Provision constraints
Ls = sparse.load_npz('data/Ls.npz').toarray()

res = []
for j in As:
    for i in j:
        if np.abs(i) > 0.0000000001:
            res.append(i)
plt.hist(res), 1 - len(res)/600**2

In [58]:

from scipy import sparse
# Q-Learning values. (Discounted Payoff action is in first column).
Qs = sparse.load_npz('data/Qs.npz').toarray()

# Association matrix
As = sparse.load_npz('data/As.npz').toarray()

# Provision constraints
Ls = sparse.load_npz('data/Ls.npz').toarray()



In [59]:
Qs_with_4_actions = Qs[:,:4]
Ls_with_4_actions = Qs[:,:4]

In [60]:
# pick 100 and check if the picked has non zero value at indices outside index subset

In [61]:
from tqdm import tqdm
bad_indices_list = []
size = 599
for seed in tqdm(range(100)):
    np.random.seed(seed)
    index_subset = set(np.random.choice(range(600), size, replace=False))
    bad_indices = 0
    for i in index_subset:
        row = As[i]
        for j, entry in enumerate(row):
            if entry > 0 and j not in index_subset:
                bad_indices += 1
                break
    bad_indices_list.append(bad_indices)

In [62]:
sorted(bad_indices_list)[0]

In [65]:
import networkx as nx
G : nx.Graph = nx.from_numpy_array(As)
connected_components = list(nx.connected_components(G))
for graph in connected_components:
    print(len(graph))


In [72]:
s_G : nx.Graph = G.subgraph(connected_components[0])
new_As = nx.to_numpy_array(s_G)

In [73]:
from ibmq_utils import generate_random_dataset

G = nx.from_numpy_array(As)
connected_components = list(nx.connected_components(G))

# Count the number of connected components
num_connected_components = len(connected_components)

# Print the result
print("Number of connected components:", num_connected_components)



In [75]:

from tew_code.qtft_tools import DC_QAOA
m = DC_QAOA(Qs_with_4_actions[:594], new_As, 1.0, 1, 1, 5)
m.set_communities()




In [81]:
(2**10) **2

Memory consumption: 134217728 bytes, 131072.00 KB, 128.00 MB
